# Part 4, Models Stacking and Validation

In [1]:
import tensorflow as tf
tf.python.control_flow_ops = tf

import pickle
import xgboost as xgb
import pandas as pd
import numpy as np
import sys
_stdout = sys.stdout

sys.path.append('modules')

import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.grid_search import GridSearchCV
from scipy.sparse import csr_matrix, hstack
from scipy.stats import iqr
from sklearn.cross_validation import KFold, train_test_split
from keras.models import Sequential
from keras.layers.advanced_activations import PReLU
from keras.layers import Dense, Dropout, Activation, BatchNormalization
from keras.callbacks import EarlyStopping
from xgboost import XGBRegressor
from xgb_regressor import XGBoostRegressor
from stacker import Stacker

%matplotlib inline

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
Using TensorFlow backend.


In the final part of this capstone, we implement a simple stacker model in which we generate predictions using two models: XGBoost and MLP. Then we combine them in a single dataset and train a second level algorithm to get the final prediction.

We are going to do a K-Fold stacking in which we train each model (level 0 model or just L0-model) on a subset of data, generate out-of-fold predictions and train the stacker (level 1 model, L1-model) on these predictions.

As usual, use can use pretrained models and skip the heavy computation phase:

In [2]:
USE_PRETRAINED = True

## Methodology

Our methodology which I took from [MLWave Ensembling Guide](http://mlwave.com/kaggle-ensembling-guide/) as well as from the winners of [Otto Group Classification Challenge](https://www.kaggle.com/c/otto-group-product-classification-challenge/forums/t/14335/1st-place-winner-solution-gilberto-titericz-stanislav-semenov) is as follows:

* **Splitting.** Split the training set into K folds


* **Out-of-fold predictions**. Fit each L0-model on K-1 folds, generate predictions for the other fold. Repeat the process for all K folds. In the end, we get predictions for the whole training set (for which we also have labels).


* **Fitting on the entire training set**. We fit each L0-model on the whole training set and get predictions for the test set. We combine predictions into a dataset, in which each feature is prediction of a single L0-model.


* **Training L1**. We fit L1-model on out-of-fold predictions, while using corresponding labels from the training set as labels for L1. After that, we ask the L1-model get the final prediction using our combine dataset of L0 predictions.

There is only one complication: we don't have the test set (we're not going to submit our prediction to Kaggle now), but we need to compare the performance of single L0-models with the stacker, as we'd like to make sure that L1-model works better than any of L0-models.

To do this, we split our training dataset into train and test subsets. We will touch the test subset only once when we compare the performance of all the models (L0 vs L1).

## Splitting

### Preparing the data
We trained our models on the same train set, but the preprocessing phase has been done differently for XGBoost and MLP. We have to replicate the same preprocessing for our ensemble phase. Let's revisit what we have to do for each model:

* **XGBoost**: 1) log-transform the target feature, 2) use a label encoding for categorical features.


* **MLP**: 1) use an one-hot encoding for categorical features.

I'll be verbose in this section and apply all the transformations manually without writing an abstraction (say, a function or a class). We only have two models and there's no need to abstract the logic in our case.

In [3]:
# XGBoost

trainxg = pd.read_csv('train.csv')
ntrain = trainxg.shape[0]

trainxg['log_loss'] = np.log(trainxg['loss'])    
features = [x for x in trainxg.columns if x not in ['id','loss', 'log_loss']]

cat_features = [x for x in trainxg.select_dtypes(
        include=['object']).columns if x not in ['id','loss', 'log_loss']]

for c in range(len(cat_features)):
    trainxg[cat_features[c]] = trainxg[cat_features[c]].astype('category').cat.codes

trainxg_x = np.array(trainxg[features])
trainxg_y = np.array(trainxg['log_loss'])

# xg_xte, xg_yte are for the final performance check
xg_xtr, xg_xte, xg_ytr, xg_yte = train_test_split(trainxg_x, trainxg_y, test_size=0.25, random_state=31337)

print "Let's see that we haven't messed up with data split:"
print "Training set X:", xg_xtr.shape, "Y:", xg_ytr.shape
print "Test set X:", xg_xte.shape, "Y:", xg_yte.shape

Let's see that we haven't messed up with data split:
Training set X: (141238, 130) Y: (141238,)
Test set X: (47080, 130) Y: (47080,)


In [4]:
# MLP

trainmlp = pd.read_csv('train.csv')
cat_names = [c for c in trainmlp.columns if 'cat' in c]

trainmlp = pd.get_dummies(data=trainmlp, columns=cat_names)

features_mlp = [x for x in trainmlp.columns if x not in ['id','loss']]

trainmlp_x = np.array(trainmlp[features_mlp])
trainmlp_y = np.array(trainmlp['loss'])

mlp_xtr, mlp_xte, mlp_ytr, mlp_yte = train_test_split(trainmlp_x, trainmlp_y, test_size=0.25, random_state=31337)

print "Let's see that we haven't messed up with data split:"
print "Training set X:", mlp_xtr.shape, "Y:", mlp_ytr.shape
print "Test set X:", mlp_xte.shape, "Y:", mlp_yte.shape

Let's see that we haven't messed up with data split:
Training set X: (141238, 1153) Y: (141238,)
Test set X: (47080, 1153) Y: (47080,)


### Preparing the L0-models

Now we just take our final models for XGBoost and MLP. Of course, we could have found more models to add to the stacker.

In [5]:
# XGBoost

reg_xgb = XGBoostRegressor(num_boost_round=200, eta=0.07, gamma=0.2, max_depth=8, min_child_weight=6,
                colsample_bytree=0.6, subsample=0.9)

In [6]:
# MLP

def hyper_model():
    model = Sequential()
    model.add(Dense(351, input_dim=len(features_mlp), init='glorot_normal'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.578947))
    
    model.add(Dense(293, init='glorot_normal'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.26666))
    
    model.add(Dense(46, init='glorot_normal'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.188888))
    
    model.add(Dense(1, init='glorot_normal'))
    model.compile(loss='mae', optimizer='adadelta')
    return model

## Out-of-fold predictions

Now we divide our training set `xg_xtr, xg_ytr` (xgboost), `mlp_xtr, mlp_ytr` (mlp) into k-folds (3), train on 2/3 folds and predict the third fold. We persist the results to files to get back to them later on.

We also save test labels (`*_test_fold_*`) to make sure that the fold generator splits data the same fashion for XGBoost and MLP and we can stack their predictions side by side.

In [7]:
# XGBoost
if not USE_PRETRAINED:
    folds = KFold(len(xg_ytr), shuffle=False, n_folds=3)

    for k, (train_index, test_index) in enumerate(folds):
        xtr = xg_xtr[train_index]
        ytr = xg_ytr[train_index]
        xte, yte = xg_xtr[test_index], xg_ytr[test_index]
        reg_xgb = XGBoostRegressor(num_boost_round=200, eta=0.07, gamma=0.2, max_depth=8, min_child_weight=6,
                        colsample_bytree=0.6, subsample=0.9)
        reg_xgb.fit(xtr, ytr)
        np.savetxt('ensemble/xgb_pred_fold_{}.txt'.format(k), np.exp(reg_xgb.predict(xte)))
        np.savetxt('ensemble/xgb_test_fold_{}.txt'.format(k), yte)

In [8]:
# MLP
if not USE_PRETRAINED:
    folds = KFold(len(mlp_ytr), shuffle=False, n_folds=3)
    for k, (train_index, test_index) in enumerate(folds):
        xtr = mlp_xtr[train_index]
        ytr = mlp_ytr[train_index]
        xte, yte = mlp_xtr[test_index], mlp_ytr[test_index]
        reg_mlp = hyper_model()
        fit = reg_mlp.fit(xtr, ytr, batch_size=128, nb_epoch=30, verbose=0)
        pred = reg_mlp.predict(xte, batch_size=256)
        np.savetxt('ensemble/mlp_pred_fold_{}.txt'.format(k), pred)
        np.savetxt('ensemble/mlp_test_fold_{}.txt'.format(k), yte)

## Training on the whole dataset

We train the same models on the whole training set (`xg_xtr, mlp_xtr` and corresponding labels) and generate predictions for the test set (`xg_xte, mlp_xte`). Remember that we do have labels for the test set, but we don't allow our L0-model see them.

In [9]:
# XGBoost
if not USE_PRETRAINED:
    reg_xgb = XGBoostRegressor(num_boost_round=200, eta=0.07, gamma=0.2, max_depth=8, min_child_weight=6,
                  colsample_bytree=0.6, subsample=0.9)
    reg_xgb.fit(xg_xtr, xg_ytr)
    np.savetxt('ensemble/xgb_pred_test.txt'.format(k), np.exp(reg_xgb.predict(xg_xte)))

In [10]:
# MLP
if not USE_PRETRAINED:
    reg_mlp = hyper_model()
    fit = reg_mlp.fit(mlp_xtr, mlp_ytr, batch_size=128, nb_epoch=30, verbose=0)
    pred = reg_mlp.predict(mlp_xte, batch_size=256)
    np.savetxt('ensemble/mlp_pred_test.txt'.format(k), pred)

## L1-model training

When the previous stage is completed, we have generated out-of-fold and test set predictions, which we can now use to train the stacker.

First, we load predictions:

In [11]:
train_xgb1 = np.loadtxt('ensemble/xgb_pred_fold_0.txt')
train_xgb2 = np.loadtxt('ensemble/xgb_pred_fold_1.txt')
train_xgb3 = np.loadtxt('ensemble/xgb_pred_fold_2.txt')

train_mlp1 = np.loadtxt('ensemble/mlp_pred_fold_0.txt')
train_mlp2 = np.loadtxt('ensemble/mlp_pred_fold_1.txt')
train_mlp3 = np.loadtxt('ensemble/mlp_pred_fold_2.txt')

### Sanity check \#1
We load labels to check that we haven't messed up with folds:

In [12]:
xgb_test_fold1 = np.exp(np.loadtxt('ensemble/xgb_test_fold_0.txt'))
xgb_test_fold2 = np.exp(np.loadtxt('ensemble/xgb_test_fold_1.txt'))
xgb_test_fold3 = np.exp(np.loadtxt('ensemble/xgb_test_fold_2.txt'))

mlp_test_fold1 = np.loadtxt('ensemble/mlp_test_fold_0.txt')
mlp_test_fold2 = np.loadtxt('ensemble/mlp_test_fold_1.txt')
mlp_test_fold3 = np.loadtxt('ensemble/mlp_test_fold_2.txt')

Recreating the original set of training set labels:

In [13]:
xgb_test_fold = np.hstack((xgb_test_fold1, xgb_test_fold2, xgb_test_fold3))
mlp_test_fold = np.hstack((mlp_test_fold1, mlp_test_fold2, mlp_test_fold3))

And testing that these labels completely match (there's still a little rounding error due to log-exp conversion):

In [14]:
mean_absolute_error(xgb_test_fold, mlp_test_fold)

1.118142715102431e-12

This is basically zero. Labels from two fold generators match and we can go on.

### Sanity check \#2

Now we load the predictions for the whole test set:

In [15]:
test_xgb = np.loadtxt('ensemble/xgb_pred_test.txt')
test_mlp = np.loadtxt('ensemble/mlp_pred_test.txt')

We should check that MAE from combined out-of-fold predictions is reasonable:

In [16]:
# Combined out-of-fold predictions for XGBoost and MLP

train_xgb_folds = np.hstack((train_xgb1, train_xgb2, train_xgb3))
train_mlp_folds = np.hstack((train_mlp1, train_mlp2, train_mlp3))

In [17]:
# MAE of XGBoost combined predictions

mean_absolute_error(np.exp(xg_ytr), train_xgb_folds)

1152.2760489657664

In [18]:
# MAE of MLP combined predictions

mean_absolute_error(mlp_ytr, train_mlp_folds)

1146.5263643278126

Predictions are around 1150-1155: this is exactly what we expect from these models.

We do the same for the test set and get scores for each single model. The score of the best single model is our baseline score for the stacker (our stacker should perform better than any given single model).

In [19]:
mae_xgb_test = mean_absolute_error(np.exp(xg_yte), test_xgb)
mae_mlp_test = mean_absolute_error(mlp_yte, test_mlp)
print "Baseline MAE which we need to improve with stacking, XGB: {}; MLP: {}.".format(mae_xgb_test, mae_mlp_test)

Baseline MAE which we need to improve with stacking, XGB: 1149.19888471; MLP: 1145.49726607.


Predictions on test should be close to 1145-1150.

### Training L1-model

Finally, it's time to join the predictions of L0-models and train a stacker over them.

In [20]:
l1_train_x = np.vstack((train_xgb_folds, train_mlp_folds)).T
l1_test_x = np.vstack((test_xgb, test_mlp)).T
l1_train_y = mlp_ytr
l1_test_y = mlp_yte

In [21]:
# Just a sanity check
print "Xtrain shape:", l1_train_x.shape
print "ytrain shape:", l1_train_y.shape
print "Xtest shape:", l1_test_x.shape
print "ytest shape:", l1_test_y.shape

Xtrain shape: (141238, 2)
ytrain shape: (141238,)
Xtest shape: (47080, 2)
ytest shape: (47080,)


The choice of the algorithm for L1 is crucial. The problem is that we don't want it overfit on the training set, and it can be very easily done. To reduce the possibility of overfitting, we should take a very simple regressor, and `LinearRegression` is the ideal candidate for us.

We now fit a very basic linear regression and get the predictions for the final test set:

In [22]:
reg = LinearRegression()

# Note that normalizing the data in case of linear models is very important
reg.fit(np.log(l1_train_x), np.log(l1_train_y))
pred = reg.predict(np.log(l1_test_x))

In [23]:
mae_stacker = mean_absolute_error(l1_test_y, np.exp(pred))

print "MAE for XGB:", mae_xgb_test
print "MAE for MLP:", mae_mlp_test
print "MAE for stacker:", mae_stacker

MAE for XGB: 1149.19888471
MAE for MLP: 1145.49726607
MAE for stacker: 1136.21813333


I got the following results:

`MAE for XGB: 1149.19888471
MAE for MLP: 1145.49726607
MAE for stacker: 1136.21813333`

We see that we improved our results dramatically. Thus, our score for a single stacker is `MAE=1136.2`.

Just to note, the coefficients of the linear combination, provided by our linear regression:

In [24]:
reg.coef_

array([ 0.59121805,  0.41705347])

Which can be interpreted this way:

PREDICTION = 0.59 \* XGB_PREDICTION + 0.41 \* MLP_PREDICTION

## Results Validation

We should validate our final model. We may encounter the problem that the model generalizes well in the specific case of our train-test split, but fails to generalize if it was trained on a different subset of data.

In this section, we train several stackers on different splits of data and make sure that all of them provide a consistent, stable performance and this performance is considerably better than the score of single L0-models.

To facilitate the process, we use mostly the same stacking technique, but I abstracted the stacker in a class `Stacker`. There are several issues to be highlighted:

* We pass functions of our models instead of the instances of models themselves. This is needed since the models should be initialized clean several times during training (e.g., for out-of-fold predictions) and I'd like to abstract the logic of the stacker class from the logic of single models' initialization.


* We may choose a different seed during stacker initialization. Thus we can shuffle the train-test split to see how our models generalize on different parts of dataset.


* The purpose of this class is solely to automate the evaluation and generalization of our stacker. This is done to be DRY and not to introduce more excessive code in this notebook.

We'll initialize the stacker on five different seeds and compare the performance.

In [25]:
seeds = [1,2,4,8,16]
final_scores = []

# Preparing XGBoost model function
xgbst = lambda _: XGBoostRegressor(num_boost_round=200, eta=0.07, gamma=0.2, max_depth=8, min_child_weight=6,
                colsample_bytree=0.6, subsample=0.9)

# Preparing MLP model function. We need to pass input dimension into
# this function, which is done in Stacker class
def hyper_model(dim):
    model = Sequential()
    model.add(Dense(351, input_dim=dim, init='glorot_normal'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.578947))
    
    model.add(Dense(293, init='glorot_normal'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.26666))
    
    model.add(Dense(46, init='glorot_normal'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.188888))
    
    model.add(Dense(1, init='glorot_normal'))
    model.compile(loss='mae', optimizer='adadelta')
    return model

We also prepare parameters for fitting MLP and predictions (amount of epochs, batch sizes) and pipeline training output to `stacker_validation.txt` file.

In [26]:
f = open('stacker_validation.txt', 'a')
_stdout = sys.stdout
sys.stdout = f

mlp_fit_params = {'nb_epoch': 30, 'batch_size': 128, 'verbose': 1}
mlp_pred_params = {'batch_size': 256, 'verbose': 1}

`stack_and_compare` returns a dictionary of final scores for both single L0-models as well as for the stacker. We initialize stackers with seeds we prepared and run stacking:

In [27]:
if not USE_PRETRAINED:
    for seed in seeds:
        stacker = Stacker(xgbst, hyper_model, train_path='train.csv', seed=seed,
                      mlp_fit_kwargs=mlp_fit_params, mlp_predict_kwargs=mlp_pred_params)
        score = stacker.stack_and_compare()
        final_scores.append(score)
        print "Seed {} completed with scores :: {}".format(seed, score)
else:
    with open('pretrained/stacker_seeds_scores.pkl', 'rb') as f:
        final_scores = pickle.load(f)

In [28]:
sys.stdout = _stdout

In [29]:
for seed, score in zip(seeds, final_scores):
    print "SEED: {}".format(seed)
    print "SCORE: {}".format(score)
    print

SEED: 1
SCORE: {'mlp': 1137.4518808596861, 'xgb': 1141.3274823341596, 'stacker': 1128.5183831793099}

SEED: 2
SCORE: {'mlp': 1138.605777626057, 'xgb': 1145.4124181185377, 'stacker': 1131.0467572871137}

SEED: 4
SCORE: {'mlp': 1139.5123466472771, 'xgb': 1145.2325510240682, 'stacker': 1130.3417696000593}

SEED: 8
SCORE: {'mlp': 1141.0668336039034, 'xgb': 1145.1971588199588, 'stacker': 1132.1655095762851}

SEED: 16
SCORE: {'mlp': 1136.586960351832, 'xgb': 1140.7780777321607, 'stacker': 1127.505237791496}



As we see, the L0-models provide very close results from seed to seed. L1-model is better in all cases: its MAE are lower, the scores are tighter clustered. To measure this fact more precisely, we calculate means and standard deviations for L0 and L1-models.

### Mean and standard deviation calculations

In [30]:
scores = pd.DataFrame(final_scores)

In [31]:
scores.describe()

,mlp,stacker,xgb
count,5.000000,5.000000,5.000000
mean,1138.644760,1129.915531,1143.589538
std,1.752011,1.889794,2.325297
min,1136.586960,1127.505238,1140.778078
25%,1137.451881,1128.518383,1141.327482
50%,1138.605778,1130.341770,1145.197159
75%,1139.512347,1131.046757,1145.232551
max,1141.066834,1132.165510,1145.412418


Observations:


* The score of all models as well as the stacker's is very consistent and stable (std are low).


* Stacker's std is lower than std of XGBoost, and just a little higher than the std of MLP. Since stacker's performance depends on the performance of single models, its tendency to keep a low standard deviation is a very good feature.


* Stacker's scores are better for each seed, its mean is lower. The worst stacker's score (1132) is still better than the best score of a single model (MLP, 1136.58).


We may now conclude that the stacker indeed outperforms any of our two models: XGBoost and MLP. We take `MAE=1129.9` (average of five stackers) as our final score.

### Improvement over baseline scores

We remember that we have the following baseline scores:

* `MAE=1217.52` — the baseline score of a Random Forest trained by Allstate competition arrangers. With our final model, we got **7.2%** score improvement.

* `MAE=1190.73` — the baseline score of a single simple model we trained (MLP).

To understand, if we got a significant improvement over those scores, let's add each of those scores to our stacker scores list and find out whether it can be called an outlier. Thus, if this baseline score can be considered an outlier, the difference between our final scores and the baseline is significant.

To run this test, we can calclucate [IQR](https://en.wikipedia.org/wiki/Interquartile_range) which is used for anomaly or outlier detection. We then calculate third quantile of data (Q3) and use the formula `Q3 + 1.5 * IQR` to set the upper score margin. Values higher than this margin are considered outliers.

In [32]:
for baseline in [1217.52, 1190.73]:
    stacker_scores = list(scores.stacker)
    stacker_scores.append(baseline)
    max_margin = np.percentile(stacker_scores, 75) + 1.5*iqr(stacker_scores)
    if baseline - max_margin > 0:
        print 'MAE =', baseline, 'is considered an outlier.'
    else:
        print 'MAE = ', baseline, 'is NOT an outlier.'

MAE = 1217.52 is considered an outlier.
MAE = 1190.73 is considered an outlier.


As we see, all baseline scores are considered outliers — we may say that our final model indeed provided a visible score improvement.